In [ ]:
!pip install web3

In [ ]:
import time
import json
import requests
import pandas as pd
from tqdm import tqdm
from web3 import Web3
from web3.exceptions import ABIFunctionNotFound

In [ ]:
ETH_API_KEY = "NATDJD6I31EU3VE55UAD1FBY3VHGF8EE27"
ETH_URL = "https://api.etherscan.io/api"

RPC_URL = "https://rpc.ankr.com/eth/b1bdd2819d3da7be4760d270af7ef985a74725cbe337c348bca633d096806c5a"
w3 = Web3(Web3.HTTPProvider(RPC_URL))

getAccountBalance_url = "https://rpc.ankr.com/multichain/79258ce7f7ee046decc3b5292a24eb4bf7c910d7e39b691384c7ce0cfb839a01/?ankr_getAccountBalance="

In [ ]:
factories = {
    "curve": {
        "address" : ["0xb9fc157394af804a3578134a6585c0dc9cc990d4",
                     "0xf18056bbd320e96a48e3fbf8bc061322531aac99",
                     "0x0c0e5f2ff0ff18a3be9b835635039256dc4b4963",
                     "0x4f8846ae9380b90d2e71d5e3d042dff3e7ebb40d"],
        "pool" : []
    }
}

In [ ]:
def get_abi(address):
    params = {
        "module": "contract",
        "action": "getabi",
        "address": address,
        "apikey": ETH_API_KEY
    }
    response = requests.post(ETH_URL, params=params)
    time.sleep(0.2)
    data = json.loads(response.text)
    result = data['result']
    return result

In [ ]:
def get_contract(address):
    abi = get_abi(address)
    checksum_address = Web3.to_checksum_address(address)
    contract = w3.eth.contract(address=checksum_address, abi=abi)
    return contract

In [ ]:
def get_fee_from_fee(contract, name):
    deno_dict = {
        "pancakeswap": 10000*100,
        "curve": 10**8,
        "uniswapv3": 10000*100
    }
    try:
        if name in deno_dict:
            fee = float(contract.functions.fee().call())
            fee = fee / deno_dict[name]
            return fee
        elif name == "fraxswap":
            fee = 10000 - float(contract.functions.fee().call())
            fee = fee / 10000
            return fee
    except Exception as e:
        print(type(e))
        return None


# NomiswapStable
def get_fee_from_swapFee(contract, name):
    try:
        swapFee = float(contract.functions.swapFee().call())
        max_fee = float(contract.functions.MAX_FEE().call())
        fee = swapFee / max_fee
        return fee
    except Exception as e:
        return None


def get_fee_from_getSwapFeePercentage(contract, name):
    if name == "balancerv2":
        swapFeePercentage = float(contract.functions.getSwapFeePercentage().call())
        fee = swapFeePercentage / 10**18
        return fee
    return None


# saddle
def get_fee_from_swapStorage(contract, name):
    if name == "saddle":
        swapStorage = contract.functions.swapStorage().call()
        swapFee = float(swapStorage[4])
        fee = swapFee / 10**8
        return fee
    return None


def get_fee_from_tradingFeePercent(contract, name):
    if name == "linkswap":
        tradingFeePercent = float(contract.functions.tradingFeePercent().call())
        fee = tradingFeePercent / 10**6
        return fee
    return None


def get_fee_from_factory(contract, name):
    if name == "crodefiswap":
        factory_address = contract.functions.factory().call()
        factory_source = get_source_code(factory_address)
        factory_contract = get_contract(factory_address, factory_source)
        totalFeeBasisPoint = float(factory_contract.functions.totalFeeBasisPoint().call())
        fee = totalFeeBasisPoint / 10000
        return fee
    return None


def get_fee(contract, name):
    fix_fee_dict = {
        "uniswapv2": 3/1000,
        "sushiswap": 3/1000,
        "verseexchange": 3/1000,
        "saitaswap": 2/1000,
        "sakeswap_pool": 3/1000,
        "xchange": 200/100000,
        "kingswap": 25/10/1000,
    }

    fee_fun = [
        get_fee_from_fee,
        get_fee_from_swapFee,
        get_fee_from_getSwapFeePercentage,
        get_fee_from_swapStorage,
        get_fee_from_tradingFeePercent,
        get_fee_from_factory
    ]

    if name in fix_fee_dict:
        return fix_fee_dict[name]

    for fun in fee_fun:
        fee = fun(contract, name)
        if fee:
            break
    return fee

In [ ]:
def get_tokens(contract):
    try:
        token0 = contract.functions.coins(0).call().lower()
        token1 = contract.functions.coins(1).call().lower()
        return token0, token1
    except Exception:
        return None, None

In [ ]:
# For curve
def get_pool_from_pool_list(factory, start, end):
    rows = []
    name = factory
    for factory_address in factories[factory]["address"]:
        factory_contract = get_contract(factory_address)
        try:
            pool_count = factory_contract.functions.pool_count().call()
            if end > pool_count:
                end = pool_count

            for i in tqdm(range(start, end)):
                try:
                    pool_address = factory_contract.functions.pool_list(i).call().lower()
                    contract = get_contract(pool_address)
                    fee = get_fee(contract, name)
                    token0, token1 = get_tokens(contract)
                    row = {
                        "name" : name,
                        "factory" : factory_address,
                        "fee" : fee,
                        "contract" : pool_address,
                        "token0" : token0,
                        "token1" : token1
                    }
                    rows.append(row)
                    time.sleep(0.2)
                except ValueError:
                    continue
        except ABIFunctionNotFound:
            continue
    pool = pd.DataFrame(rows)
    pool = pool.drop_duplicates(ignore_index = True)
    return pool

In [ ]:
def get_tvl(pool):
    token_type = ['NATIVE']
    rows = []

    for i in tqdm(range(0, len(pool))):
        payload = {
            "jsonrpc": "2.0",
            "method": "ankr_getAccountBalance",
            "params": {
                "blockchain": ["eth"],
                "walletAddress": pool.contract[i],
                "nativeFirst": True,
                "onlyWhitelisted": True,
                "pageSize": 2
            },
            "id": 1
        }
        headers = {
            "accept": "application/json",
            "content-type": "application/json"
        }

        response = requests.post(getAccountBalance_url, json=payload, headers=headers)
        time.sleep(1)
        if response.status_code == 200:
            try:
                data = json.loads(response.text)
                tvl = data['result']['totalBalanceUsd']
            except KeyError:
                tvl = None
            row = {
                "name" : pool.name[i],
                "factory" : pool.factory[i],
                "tvl" : tvl,
                "fee" : pool.fee[i],
                "contract" : pool.contract[i],
                "token0" : pool.token0[i],
                "token1" : pool.token1[i],
            }
            rows.append(row)
    pool = pd.DataFrame(rows)
    pool = pool.dropna(axis=0).reset_index(drop=True)
    return pool

In [ ]:
for count in range(0, 10000):
    start = count * 1000
    end = start + 1000
    
    pool = get_pool_from_pool_list("curve", start, end)
    if len(pool) <= 0:
        break
    file_name = f"part{count}.csv"
    pool.to_csv(file_name, index=False)
        
    pool = get_tvl(pool)
    file_name = f"part{count}.csv"
    pool.to_csv(file_name, index=False)

In [ ]:
files = [f"part{i}.csv" for i in range(count)]

In [ ]:
combined_df = pd.DataFrame()

In [ ]:
for file in files:
    df = pd.read_csv(file)
    combined_df = pd.concat([combined_df, df], ignore_index=True)
    
combined_df = combined_df.drop_duplicates()
combined_df

In [ ]:
combined_df.to_csv('eth_curve.csv', index=False)